In [ ]:
# Install necessary packages
!pip python3 -m pip install googlesearch-python
!pip3 install pytube
!pip install mutagen
!pip install pydub


ERROR: unknown command "python3"
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 2.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 KB 6.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"

/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force&state=xuxrjBKKbIYmIL1LZH-ECksdMehPsSwTxpEBwjrev8I'
/bin/sh: 1: firefox: not found
/bin/sh: 1: google-chrome: not found
/bin/sh: 1: chromium-browser: not found
/bin/sh: 1: open: not found
Cannot retrieve auth tokens.
Failure("Error opening URL:https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.

In [ ]:
# Import necessary packages
from google.colab import drive
from googlesearch import search
from pytube import YouTube
#from 6 import AudioSegment
import os
import math
# import required modules
import subprocess
import mutagen
from mutagen.wave import WAVE
#split video into chunks
from pydub import AudioSegment
from pydub.utils import make_chunks
import os
import pandas as pd
import itertools

In [ ]:
#Main Directory
DIR = "/content/gdrive/My Drive/Meta_Asr/Data_Annotation/Audio_Data/"
TEMP_FILENAME = "/content/Audio_Data/"
AUDIO_WAV_FILENAME=DIR+"Audio_Wav/"




#configuration setup
def setup(filename):
  try:
    os.mkdir(filename)
  except OSError:
    pass

# Function to search for YouTube videos based on keywords
def search_youtube_videos(keywords,n):
    urls = []
    query = f"site:youtube.com/watch {keywords}"
    for url in search(query,stop=n):
        urls.append(url)
    return urls



#convert video to mp3
def video_to_mp3(lien_vid,filename=TEMP_FILENAME):
  setup(filename)
  # url input from user
  yt = YouTube(str(lien_vid))
  # extract only audio
  video = yt.streams.filter(only_audio=True).first()
  # check for destination to save file
  print("Enter the destination (leave blank for current directory)")
  destination = filename
  # download the file
  out_file = video.download(output_path=destination)
  # save the file
  base, ext = os.path.splitext(out_file)
  new_file = base + '.mp3'
  os.rename(out_file, new_file)
  # result of success
  print(yt.title + " has been successfully downloaded.")


# convert a audio.mp3 to audio.wav
def convert_audio(audio_wav_filename=AUDIO_WAV_FILENAME,video_name="video_audio{0}"):
  filename=audio_wav_filename
  setup(audio_wav_filename)
  for (root,dirs,files) in os.walk(TEMP_FILENAME, topdown=True):
    for i  in range(len(files)):
      wavfile_name=filename+video_name.format(i)+'.wav'
      audio=os.path.join(TEMP_FILENAME,files[i])
      subprocess.call(['ffmpeg','-i',audio,os.path.join(audio_wav_filename,wavfile_name)])


# function to convert the information into
# some readable format
def extrcat_time(length):
	hours = length // 3600 # calculate in hours
	length %= 3600
	mins = length // 60 # calculate in minutes
	length %= 60
	seconds = length # calculate in seconds
	return hours, mins, seconds # returns the duration

#compute the duration audio
def audio_duration(audio):
  audio = WAVE(audio)
  # contains all the metadata about the wavpack file
  audio_info = audio.info
  length = int(audio_info.length)
  hours, mins, seconds = extrcat_time(length)
  return seconds

#split each audio into chunks
def split_into_chunks(audio,j,ms=10000,chunks_filename=DIR+"chunks_Audio_Wav_video{0}/",chunks_name="chunk{0}.wav"):
  data=list()
  filename=chunks_filename.format(j)
  setup(filename)
  myaudio = AudioSegment.from_file(audio , "wav")
  chunk_length_ms =ms  # pydub calculates in millisec
  chunks = make_chunks(myaudio,chunk_length_ms)
  #Make chunks of one sec
  #Export all of the individual chunks as wav files
  for  i in range(len(chunks)):
    chunk_name = os.path.join(filename,chunks_name.format(i))
    print(chunk_name)
    chunks[i].export(chunk_name, format="wav")
    data.append(chunk_name)

  return data


def create_audio_data(keywords,n):
  #config setup
  setup(DIR)
  list_audio = list()
  #extract vid links
  urls = search_youtube_videos(keywords,n)
  #convert to mp3
  for link in urls:
    video_to_mp3(link)
  #convert to .wav
  convert_audio()
  #split to chunks
  for (root,dirs,files) in os.walk(AUDIO_WAV_FILENAME,topdown=True):
    for j in range(len(files)):
      data=split_into_chunks(AUDIO_WAV_FILENAME+files[j],j)
      list_audio.append(data)
  audio_data =list(itertools.chain.from_iterable(list_audio))
  #create dataframe
  df = pd.DataFrame(audio_data,columns=["Audio"])
  #save in csv file
  df.to_csv(DIR+"data_audio.csv")

  return df













In [ ]:
if __name__ == '__main__':
  keywords = "sexcual abuse kids real story tell"
  n=15
  df=create_audio_data(keywords,n)




Enter the destination (leave blank for current directory)
Child Sexual Abuse - A Woman's True Story has been successfully downloaded.
Enter the destination (leave blank for current directory)
Survivors of child violence tell their stories - Full version has been successfully downloaded.
Enter the destination (leave blank for current directory)
The Horrors of Child Sexual Abuse - A Survivor Tells His Story - WRAL Podcast Episode 1 has been successfully downloaded.
Enter the destination (leave blank for current directory)
'Leaving Neverland' tells disturbing stories of child sex abuse has been successfully downloaded.
Enter the destination (leave blank for current directory)
50. MY STORY OF CHILD SEXUAL ABUSE has been successfully downloaded.
Enter the destination (leave blank for current directory)
Abuse Story has been successfully downloaded.
Enter the destination (leave blank for current directory)
On the Level: Children Being Sexually Abused (Documentary) has been successfully downlo